# Calenvironscore, Hazardous Waste Sites, and Schools by County

This is some background information about the project

In [103]:
import geopandas as gpd
from ipyleaflet import Map, GeoData, basemaps, LayersControl, GeoJSON, LegendControl
import json
import ipywidgets
from ipywidgets import Layout
from matplotlib.colors import to_hex
import matplotlib.pyplot as plt
from functools import partial

In [104]:
def handle_new_var(change):
    cal_layer.style_callback = partial(feature_color,var = change.new)


In [105]:
temps = ['#009392','#39b185','#9ccb86','#e9e29c','#eeb479','#e88471','#cf597e']
sunset = ['#f3e79b','#fac484','#f8a07e','#eb7f86','#ce6693','#a059a0','#5c53a5']
purpor = ['#f9ddda','#f2b9c4','#e597b9','#ce78b3','#ad5fad','#834ba0','#573b88']

In [106]:
lookup_table = {'CIscoreP':[  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'OzoneP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'PM2_5_P': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'DieselPM_P':[  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'PesticideP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'Tox_Rel_P': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'TrafficP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'DrinkWatP':[  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'Lead_P': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'CleanupP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'GWThreatP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'HazWasteP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'ImpWatBodP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'SolWasteP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'PolBurdP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'AsthmaP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'LowBirWP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'CardiovasP':[  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'EducatP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'Ling_IsolP':[  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'PovertyP':[  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'UnemplP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'HousBurdP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'PopCharP': [  0.        ,  14.28571429,  28.57142857,  42.85714286,
         57.14285714,  71.42857143,  85.71428571, 100.        ],
 'Hispanic': [ -1.      ,  10.414156,  16.993808,  25.102892,  37.262714,
         53.427914,  71.86412 , 100.      ],
 'White': [ -1.      ,   7.329732,  16.848244,  29.085772,  43.918998,
         57.28446 ,  70.26504 , 100.      ],
 'AfricanAm': [-1.      ,  0.281864,  0.978956,  1.886408,  3.326938,  5.691058,
        10.6181  , 84.7082  ],
 'NativeAm': [ -1.      ,   0.      ,   0.      ,   0.      ,   0.      ,
          0.283014,   0.66225 , 100.      ],
 'OtherMult': [-1.      ,  0.652344,  1.572924,  2.366828,  3.320956,  4.354834,
         5.76382 , 17.0653  ],
 'BIPOC': [0.        , 0.21473146, 0.31803276, 0.45102775, 0.61592223,
        0.77472911, 0.9006613 , 1.        ],
 'CleanupSite_Density': [1.31506805e-04, 1.22203901e-01, 3.35259215e-01, 5.25072014e-01,
        7.74143341e-01, 1.12275009e+00, 1.93305654e+00, 3.83504085e+01]
}


In [107]:
region_focus = {'Santa Barbara': ([34.67, -120.02], 10),
 'Colusa': ([39.18, -122.24], 10),
 'Tehama': ([40.13, -122.23], 9),
 'Imperial': ([33.04, -115.37], 9),
 'Monterey': ([36.22, -121.24], 9),
 'Amador': ([38.45, -120.65], 10),
 'Placer': ([39.06, -120.75], 10),
 'Calaveras': ([38.2, -120.55], 10),
 'Mono': ([37.94, -118.89], 9),
 'Orange': ([33.7, -117.76], 10),
 'Sonoma': ([38.53, -122.89], 9),
 'Siskiyou': ([41.59, -122.54], 9),
 'Kern': ([35.34, -118.73], 9),
 'Yolo': ([38.69, -121.9], 10),
 'Sacramento': ([38.45, -121.34], 10),
 'Madera': ([37.22, -119.76], 9),
 'Mendocino': ([39.44, -123.39], 9),
 'Tulare': ([36.22, -118.8], 9),
 'Yuba': ([39.27, -121.35], 10),
 'Plumas': ([40.0, -120.84], 10),
 'Stanislaus': ([37.56, -121.0], 10),
 'Shasta': ([40.76, -122.04], 9),
 'Fresno': ([36.76, -119.65], 9),
 'Marin': ([38.07, -122.72], 10),
 'Mariposa': ([37.58, -119.91], 10),
 'Alpine': ([38.6, -119.82], 10),
 'Glenn': ([39.6, -122.39], 10),
 'Lassen': ([40.67, -120.59], 9),
 'Del Norte': ([41.74, -123.9], 10),
 'San Mateo': ([37.42, -122.33], 10),
 'San Luis Obispo': ([35.39, -120.4], 9),
 'Napa': ([38.51, -122.33], 10),
 'San Joaquin': ([37.93, -121.27], 10),
 'Nevada': ([39.3, -120.77], 10),
 'San Bernardino': ([34.84, -116.18], 8),
 'Riverside': ([33.74, -115.99], 9),
 'Sutter': ([39.03, -121.69], 10),
 'Trinity': ([40.65, -123.11], 9),
 'Contra Costa': ([37.92, -121.93], 10),
 'Ventura': ([34.46, -119.08], 9),
 'El Dorado': ([38.77, -120.54], 10),
 'Tuolumne': ([38.03, -119.95], 10),
 'Sierra': ([39.58, -120.52], 10),
 'Lake': ([39.1, -122.75], 10),
 'Butte': ([39.67, -121.6], 10),
 'Modoc': ([41.59, -120.72], 9),
 'Alameda': ([37.65, -121.89], 10),
 'San Francisco': ([37.76, -122.44], 10),
 'Merced': ([37.19, -120.72], 10),
 'Inyo': ([36.51, -117.41], 8),
 'Solano': ([38.27, -121.93], 10),
 'Kings': ([36.08, -119.82], 10),
 'Humboldt': ([40.7, -123.88], 8),
 'Los Angeles': ([34.32, -118.22], 9),
 'Santa Clara': ([37.18, -121.75], 5),
 'San Diego': ([32.99, -116.85], 9),
 'San Benito': ([36.59, -121.09], 9),
 'Santa Cruz': ([37.02, -121.97], 10),               
               }


In [108]:
def handle_new_region(change):
    center,zoom = region_focus[change.new]
    m.center = center
    m.zoom = zoom
    cal_reduced = calenviroscreen[calenviroscreen['County'] == change.new]
    school_reduced = schools[schools['County'] == change.new]
    hzw_sites_reduced = hzw_sites[hzw_sites['COUNTY'] == change.new.upper()]

    cal_layer.data = cal_reduced.__geo_interface__
    school_layer.data = school_reduced.__geo_interface__
    hzw_sites_layer.data = hzw_sites_reduced.__geo_interface__

In [109]:
var_caption = ipywidgets.Label(value='Choose a variable to display as color on the map')
var_selection = ipywidgets.Dropdown(
    options=[('CalEnviroScreen Score','CIscoreP'),
             ('Linguistic Isolation','Ling_IsolP'),
             ('Pesticide Use','PesticideP'),
             ('% BIPOC', 'BIPOC'),
             ('Density of Cleanup Sites','CleanupSite_Density'),
            ],
    value='CIscoreP',
    description='Variable:',
)
var_selection.observe(handle_new_var,names='value')
display(var_caption, var_selection)

Label(value='Choose a variable to display as color on the map')

Dropdown(description='Variable:', options=(('CalEnviroScreen Score', 'CIscoreP'), ('Linguistic Isolation', 'Li…

In [110]:
region_caption = ipywidgets.Label(value='Choose a county to display')
region_selection = ipywidgets.Dropdown(
    options= sorted(list(region_focus.keys())),
    value='Los Angeles',
    description='County:',
)
region_selection.observe(handle_new_region, names='value')
display(region_caption, region_selection)

Label(value='Choose a county to display')

Dropdown(description='County:', index=18, options=('Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colus…

In [111]:
calenviroscreen = gpd.read_file("calenviroscreen.gpkg")
calenviroscreen_la = calenviroscreen[calenviroscreen['County'] == 'Los Angeles']


In [112]:
def feature_color(feature, var='CIscoreP'):
    score = feature['properties'][var]
    breaks = lookup_table.get(var)
    if var.endswith('P') or var =='CleanupSite_Density':
        colormap = temps
    else:
        colormap = purpor
    if score < 0:
        return {'fillColor':'gray','color':'gray'}
    else:
        for i,breakpoint in enumerate(breaks):
            if score <= breakpoint:
                break
    return {'fillColor':colormap[i-1], 'color':colormap[i-1]}

In [113]:

def feature_color_percent(feature, var='CIscoreP'):
    #print(feature['properties'])
    score = feature['properties'][var]
    if var.endswith('P'):

        if score < 0:
            region_color = 'gray' # NaNs
        elif score < 25:
            region_color = 'green'
        elif score < 50:
            region_color = 'yellow'
        elif score < 75:
            region_color = 'orange'
        else:
            region_color = 'red'
    return {'fillColor': region_color, 'color': region_color}


In [114]:
m = Map(center=(33.94, -118.24), zoom = 10, basemap= basemaps.Esri.WorldTopoMap,
       layout=Layout(height='600px'))
cal_layer = GeoJSON(data = calenviroscreen_la.__geo_interface__,
                    style={'weight':0.1,'fillOpacity':0.2},
                    hover_style={'fillOpacity': 0.4},
                    #style_callback=lambda feat: {"opacity": 0}, #This was an attempt to stop the layer from blinking as we switch. It did not work
                    name = 'CalEnvironScreen')
m.add_layer(cal_layer)
cal_layer.style_callback = partial(feature_color, var='CIscoreP')

m

Map(center=[33.94, -118.24], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [115]:
hzw_sites = gpd.read_file(f'hzw_sites.gpkg')

In [116]:
schools = gpd.read_file('schools.gpkg')

In [117]:
schools_la = schools[schools['County']=='Los Angeles']
hzw_sites_la = hzw_sites[hzw_sites['COUNTY']=='LOS ANGELES']

In [118]:
school_layer =  GeoJSON(data = schools_la.__geo_interface__,
                    style={'color': 'gold', 'radius':8, 'fillColor': 'gold', 'opacity':0.9, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.7},
                    hover_style={'fillColor': 'gold' , 'fillOpacity': 0.7},
                    name = 'Schools')

In [119]:
m.add_layer(school_layer)


In [120]:
hzw_sites_layer =  GeoJSON(data = hzw_sites_la.__geo_interface__,
                    style={'color': 'red', 'radius':2, 'fillColor': 'red', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.3},
                    #hover_style={'fillColor': 'red' , 'fillOpacity': 0.7},
                    point_style={'radius': 2, 'color': 'red', 'fillOpacity': 0.3, 'fillColor': 'red', 'weight': 0},
                    name = 'Hazardous Waste Cleanup Sites')

In [121]:
m.add_layer(hzw_sites_layer)

In [122]:
control = LayersControl(position='topright')
m.add_control(control)


In [123]:
def update_html(feature, **kwargs):
    #print(feature['properties'])
    tract = feature['properties']['Tract']
    hover_info.value = f'''
    <h3>Census Tract Information</h3>
        <h4><b>Tract #: {tract}</b></h4>
        Location: {feature['properties']['ApproxLoc']}</br>
        Population: {feature['properties']['TotPop19']}</br>
    '''
    df = hzw_sites[hzw_sites['Tract'] == tract]
    site_info.clear_output()
    with site_info:
        display(df)


In [124]:
site_info = ipywidgets.Output()
display(site_info)

Output()

In [125]:
hover_info = ipywidgets.HTML(
    value="""
    <h3>Census Tract Information</h3>
    Hover over a region
    """,
)
display(hover_info)

HTML(value='\n    <h3>Census Tract Information</h3>\n    Hover over a region\n    ')

In [126]:
cal_layer.on_hover(update_html)

In [127]:
legend = LegendControl({"Hazardous Waste Sites":"red", "Schools":"Gold"}, name="Legend", position="bottomright")
m.add_control(legend)
